# Deep learning for Star-Galaxy separation
https://arxiv.org/pdf/1608.04369.pdf


Most existing star-galaxy classifiers require careful feature extraction and selection. The latest advances in machine learning that use deep convolutional neural networks allow a machine to automatically learn the features directly from data, minimizing the need for input from human experts. In these lab we present a star-galaxy classification framework that uses deep convolutional neural networks to solve this problem

<img src="https://old.ipac.caltech.edu/2mass/releases/spr99/doc/test/jarrett2/old/star_gal/jhk_lowdensity.gif" style="width: 400px;"/>

# Section 1: Setup

### Import libraries

In [1]:

import matplotlib as mpl
mpl.use('Agg')
%matplotlib inline
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

#from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout,  Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
#from keras.layers.noise import GaussianNoise
#from keras.models import load_model
from keras import backend as K
K.set_image_dim_ordering('th')
#from keras.callbacks import EarlyStopping
#from keras.callbacks import ModelCheckpoint
#from keras.utils import np_utils
from keras.constraints import maxnorm
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

Using TensorFlow backend.


### SET PATH
Set the path to the folder containing the inputs datasets:

In [2]:
#=====================================================================
# (0.1) CHOOSE SETTING
#=====================================================================
#pathinData = '/home/diego/catalogs/dataNumpyForm/'
#Y_sim = np.load(pathinData+'ParametersSimulated_ALL.npy')
#X_sim = np.load(pathinData+'SimulatedStamps_ALL.npy')

pathinData = '/home/diego/catalogs/dataNumpyForm/SDSS/'
X_tot = np.load(pathinData+'StarGalaxy_Images1.npy')
Y_cat =  pd.read_pickle(pathinData+'StarGalaxy_pandas1')

FileNotFoundError: [Errno 2] No such file or directory: '/home/diego/catalogs/dataNumpyForm/SDSS/StarGalaxy_Images1.npy'

# Section 2: Load Data

I've saved the data that you will use in this tutorial in numpy format. The data consist of ...., The class that we aim to predict is stored in the target file Y. 

In [ ]:
Y_cat.loc[Y_cat['class'] == 'STAR', 'class'] = 1.
Y_cat.loc[Y_cat['class'] == 'QSO', 'class'] = 1.
Y_cat.loc[Y_cat['class'] == 'GALAXY', 'class'] = 0.



x = Y_cat.as_matrix(['class'])



Y_tot = []
for i in range (len(x)):
    if x[i] == 0:
        Y_tot = np.append(Y_tot,0.)
    else:
        Y_tot = np.append(Y_tot,1.)
    

        

X_tot = np.moveaxis(X_tot, 3, 1)



In [ ]:
# Spliting in Training and Test datasets
X_train = X_tot[0:len(X_tot)//5*4,:,:,:]   
Y_train = Y_tot[0:len(Y_tot)//5*4,]
X_val = X_tot[len(X_tot)//5*4:,:,:,:]
Y_val = Y_tot[len(Y_tot)//5*4:,]

# Section 3: Build the CNN models

In [ ]:
#====================================================================== 
# (3) MODEL MODULEs
#=====================================================================
#=========================================================================  
#BUILT Model
#================================================ 
#======================

def Model1():
    #Parameter Network
    img_rows=400
    img_cols=400
    dropoutpar=0.15
    img_channels=3
    depth=16
    #model
    model = Sequential()
    model.add(Convolution2D(depth, 3, 3,init='normal',activation='relu', W_constraint=maxnorm(3),border_mode='same', input_shape=(img_channels,img_rows, img_cols)))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Convolution2D(depth*2, 3, 3,activation='relu', W_constraint=maxnorm(3),border_mode='same'))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Convolution2D(depth*4, 3, 3,activation='relu', W_constraint=maxnorm(3),border_mode='same'))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu', W_constraint=maxnorm(3)))
    model.add(Dropout(dropoutpar)) 
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    return model
    
    
def Model2():
    img_rows=128
    img_cols=128
    img_channels=1
    depth=16
    dropoutpar=0.15
    #model
    model = Sequential()
    model.add(Convolution2D(depth, 3, 3,init='normal',activation='relu', W_constraint=maxnorm(3),border_mode='same', input_shape=(img_channels, img_rows, img_cols)))
    model.add(Convolution2D(depth*2, 3, 3,activation='relu', W_constraint=maxnorm(3),border_mode='same'))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Convolution2D(depth*4, 3, 3,activation='relu', W_constraint=maxnorm(3),border_mode='same'))
    model.add(Convolution2D(depth*4, 3, 3,activation='relu', W_constraint=maxnorm(3),border_mode='same'))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Convolution2D(depth*4, 3, 3,activation='relu', W_constraint=maxnorm(3),border_mode='same'))
    model.add(Convolution2D(depth*4, 3, 3,activation='relu', W_constraint=maxnorm(3),border_mode='same'))
    model.add(Convolution2D(depth*4, 3, 3,activation='relu', W_constraint=maxnorm(3),border_mode='same'))
    model.add(Convolution2D(depth*4, 3, 3,activation='relu', W_constraint=maxnorm(3),border_mode='same'))
    model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Flatten())
    model.add(Dense(64, activation='relu', W_constraint=maxnorm(3)))
    model.add(Dropout(dropoutpar))
    model.add(Dense(32, activation='relu', W_constraint=maxnorm(3)))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    return model
    

# Section 4: Build a training module

In [ ]:
#==================
# FIT MODEL
#==================
def Fit_Model(X_train, X_val,  Y_train, Y_val,  model):   
    batch_size=64
    nb_epoch = 4    
    lr=0.01 
    decay=0   
    momentum=0.9 
    
    sgd = SGD(lr=lr, decay=decay, momentum=momentum, nesterov=True)
    model.compile(loss='binary_crossentropy',optimizer=sgd,metrics=['accuracy'])
  
        
    #data argumentation    
    datagen = ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            rotation_range = 0., #360
            width_shift_range=0.05, #0.05  
            height_shift_range=0.05, #0.05 
            horizontal_flip=False,
            vertical_flip=False,
 	    zoom_range=0.) #[0.7,1.3] 
    
    #Initialize variables
    train_loss_vec=[]
    test_loss_vec=[]
    maxPatience = 10
    consumedPatience =0
    test_loss = 999.
    best_loss = 999.
    epochsDone = 0
    bestModel=[]                             
                                                                  
    for e in range(1, nb_epoch):
        epochsDone = epochsDone+1
        train_loss = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=1)
        pred=model.predict_classes(X_val)
        test_loss = 1./accuracy_score(Y_val, pred)           
        #train_loss_vec = np.append(train_loss_vec,train_loss.history['loss']) 
        #test_loss_vec = np.append(test_loss_vec,test_loss)
                
        #implementation early stopping
        print ('Validation loss = ', test_loss)
        print (' ')         
        if test_loss < best_loss:
            best_loss = test_loss
            bestModel = model
            consumedPatience = 0
            print ('Here I save best model and best loss')
        else:            
            consumedPatience = consumedPatience + 1
        if consumedPatience == maxPatience:
           break  
              
    return bestModel, epochsDone      

    


# Section 5: Train and Validate

In [ ]:
model =Model1()
model.summary()
print('-'*30)
print('Fitting model...')
print('-'*30)
model, epochsDone = Fit_Model(X_train, X_val, Y_train, Y_val, model)  
#then SAVE the MODEL 
model.save(pathinModel+'Model.h5')

In [ ]:
pred=model.predict_classes(X_val)
print(classification_report(pred,Y_val))

from sklearn.metrics import accuracy_score
print(accuracy_score(Y_val, pred))

Notas: Early Stopping, Best model, qué es lo mejor que puedo esperar?, pedirles que jueguen con los parametros, darles algunas imagenes de test para probar?, por qué dos modelos? les damos solo uno y pistas para el segundo?, Por qué es importante el problema?, darles el summary y que hagan ellos la arquitectura
 Necesitamos el train_loss_vec y demás?   